In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import sys
import os
import glob
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.magopter as mg
import flopter.core.constants as c
import flopter.databases.magnum as mdb
import flopter.core.normalisation as nrm
import flopter.utils.lputils as lp
import flopter.classes.ivdata as ivd
import flopter.core.filtering as filt
import scipy.signal as sig
import flopter.core.fitters as fts

**FILE IO**

In [2]:
folders = ['2019-05-28_Leland/', '2019-05-29_Leland/']
mg.Magopter._FOLDER_STRUCTURE = '/Data/external/magnum/'
files = []
file_folders = []
for folder1 in folders:

    os.chdir(mg.Magopter.get_data_path() + folder1)
    files.extend(glob.glob('*.adc'))
    file_folders.extend([folder1] * len(glob.glob('*.adc')))
files.sort()

# for i, f in enumerate(files):
#     print(i, f)

# file = files[286]
# adc_file = files[285]
# ts_file = files[284]
adc_file = files[-1]
folder = folders[-1]
print('"{}" \t\t "{}"'.format(folder, adc_file))


mp = lp.MagnumProbes()
probe_S = mp.probe_s
probe_L = mp.probe_l

0 2019-05-28 10h 30m 08s TT_06695992307612752513.adc
1 2019-05-28 12h 14m 30s TT_06696019202114007511.adc
2 2019-05-28 12h 21m 44s TT_06696021067838019493.adc
3 2019-05-28 12h 48m 38s TT_06696027997221515869.adc
4 2019-05-28 14h 20m 45s TT_06696051736256851667.adc
5 2019-05-28 15h 49m 45s TT_06696074692831398075.adc
6 2019-05-28 15h 56m 35s TT_06696076438102672256.adc
7 2019-05-28 15h 58m 31s TT_06696076936923400130.adc
8 2019-05-28 16h 19m 55s TT_06696082451310700227.adc
9 2019-05-28 16h 22m 11s TT_06696082985479260440.adc
10 2019-05-28 16h 25m 58s TT_06696084010909391205.adc
11 2019-05-28 16h 27m 39s TT_06696084446035357724.adc
12 2019-05-28 16h 31m 10s TT_06696085352966572043.adc
13 2019-05-28 16h 32m 42s TT_06696085746216338023.adc
14 2019-05-28 17h 14m 14s TT_06696096449516280973.adc
15 2019-05-28 17h 16m 07s TT_06696096934807785572.adc
16 2019-05-28 17h 18m 00s TT_06696097420614687980.adc
17 2019-05-28 17h 20m 21s TT_06696098027540441792.adc
18 2019-05-28 17h 22m 35s TT_066960986

In [141]:
dsr = 10

# Create magopter object
magopter = mg.Magopter(folder, adc_file, shunt_resistor=10.0, cabling_resistance=6)
magopter._VOLTAGE_CHANNEL = 3
magopter._PROBE_CHANNEL_3 = 4
magopter._PROBE_CHANNEL_4 = 5
magopter.prepare(down_sampling_rate=dsr, roi_b_plasma=True, filter_arcs_fl=False, crit_freq=None, crit_ampl=None)

print('0: {}, 1: {}'.format(len(magopter.iv_arrs[0]), len(magopter.iv_arrs[1])))



Start Time:  17:28:48
End Time:  17:29:28
Beam Down Timestamp:  6696471324787694560 1559143728.76
Regular Timestamp:  6696471324809390982 1559143728.7650516


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


0: 1998, 1: 1998


In [4]:
# Get values for TS measurements, depending on whether Magnum database is available.
magnum_access_fl = magopter.ts_temp is not None
if magnum_access_fl:
    temps = [np.max(temp) / nrm.ELEM_CHARGE for temp in magopter.ts_temp[mdb.DATA]]
    denss = [np.max(dens) for dens in magopter.ts_dens[mdb.DATA]]
    T_e_ts = np.mean(temps)
    d_T_e_ts = np.std(temps) / np.sqrt(len(temps))
    n_e_ts = np.mean(denss)
    d_n_e_ts = np.std(denss) / np.sqrt(len(denss))
else:
    T_e_ts = 1.12
    d_T_e_ts = 0.01
    n_e_ts = 4.44e20
    d_n_e_ts = 0.01e20
#     T_e_ts = 2.24
#     d_T_e_ts = 0.01
#     n_e_ts = 1.14e19
#     d_n_e_ts = 0.01e19
#     T_e_ts = 1.61
#     d_T_e_ts = 0.01
#     n_e_ts = 1.4e20
#     d_n_e_ts = 0.1e20

In [6]:
print(magopter.arcs)

[]


In [30]:
fit_dfs = magopter.fit(load_fl=True)

**Xarray dataset creation + plotting**

In [52]:
# print length of the 0th probes (probe S) number of sweeps
print(len(magopter.iv_arrs[1]))

# Create relative t array by subtracting the first timestep value from the first time array 
first_time_arr = magopter.iv_arrs[1][0]['t']
relative_t = np.zeros(len(first_time_arr))

sweep_length = np.shape(relative_t)[0] // 2
print(sweep_length)
# create a list of datasets for each sweep
probe_designations = ['S', 'L']
ds_probes = []

for i in range(len(magopter.iv_arrs)):
    ds_list = []
    for j, iv in enumerate(magopter.iv_arrs[i]):
        if j % 2 == 0:
            ds = xr.Dataset({'voltage': (['time'], iv['V'][:sweep_length]),
                             'current': (['time'], iv['I'][:sweep_length]),
                             'shot_time': (['time'], iv['t'][:sweep_length]),
                             'start_time': iv['t'][0]},
                            coords={'time': relative_t[:sweep_length], 'direction': 'up', 'probe':probe_designations[i]})
        else:
            ds = xr.Dataset({'voltage': (['time'], np.flip(iv['V'][:sweep_length])),
                             'current': (['time'], np.flip(iv['I'][:sweep_length])),
                             'shot_time': (['time'], np.flip(iv['t'][:sweep_length])),
                             'start_time': iv['t'][0]},
                            coords={'time': relative_t[:sweep_length], 'direction': 'down', 'probe':probe_designations[i]})
        ds_list.append(ds)

    # # Separate into up and down sweeps then concat along sweep direction as an axis
    print(len(ds_list[::2]), len(ds_list[1::2]))
    ds_ups = xr.concat(ds_list[::2], 'sweep')
    ds_downs = xr.concat(ds_list[1::2], 'sweep')

    direction = xr.DataArray(np.array(['up', 'down']), dims=['direction'], name='direction')
    ds_probes.append(xr.concat([ds_ups, ds_downs], dim=direction))

probe = xr.DataArray(np.array(probe_designations), dims=['probe'], name='probe')
ds_full = xr.concat(ds_probes, dim=probe)

ds_full = ds_full.sel(probe = 'S')

manual_start = 0
manual_end = 750
if not magnum_access_fl:
    ds_full.max(dim='time').mean('direction')['current'].plot.line(x='sweep')
    ds_full.max(dim='time').mean('direction').isel(sweep=slice(manual_start,manual_end))['current'].plot.line(x='sweep')

    # Choose only the IVs in the static section
    ds_full = ds_full.isel(sweep=slice(manual_start,manual_end))

# Average across each sweep direction
sweep_avg_up = ds_full.sel(direction = 'up').mean('sweep')
sweep_avg_dn = ds_full.sel(direction = 'down').mean('sweep')

# Add in standard deviation of each bin as a new data variable 
sweep_avg_up = sweep_avg_up.assign({'d_current': ds_full.sel(direction = 'up').std('sweep')['current']})
sweep_avg_dn = sweep_avg_dn.assign({'d_current': ds_full.sel(direction = 'down').std('sweep')['current']})

print(ds_full)
relative_t = first_time_arr - first_time_arr[0]

1998
500


In [53]:
# create a list of datasets for each sweep
probe_designations = ['S', 'L']
ds_probes = []

for i in range(len(magopter.iv_arrs)):
    ds_list = []
    for j, iv in enumerate(magopter.iv_arrs[i]):
        if j % 2 == 0:
            ds = xr.Dataset({'voltage': (['time'], iv['V'][:sweep_length]),
                             'current': (['time'], iv['I'][:sweep_length]),
                             'shot_time': (['time'], iv['t'][:sweep_length]),
                             'start_time': iv['t'][0]},
                            coords={'time': relative_t[:sweep_length], 'direction': 'up', 'probe':probe_designations[i]})
        else:
            ds = xr.Dataset({'voltage': (['time'], np.flip(iv['V'][:sweep_length])),
                             'current': (['time'], np.flip(iv['I'][:sweep_length])),
                             'shot_time': (['time'], np.flip(iv['t'][:sweep_length])),
                             'start_time': iv['t'][0]},
                            coords={'time': relative_t[:sweep_length], 'direction': 'down', 'probe':probe_designations[i]})
        ds_list.append(ds)

    # # Separate into up and down sweeps then concat along sweep direction as an axis
    print(len(ds_list[::2]), len(ds_list[1::2]))
    ds_ups = xr.concat(ds_list[::2], 'sweep')
    ds_downs = xr.concat(ds_list[1::2], 'sweep')

    direction = xr.DataArray(np.array(['up', 'down']), dims=['direction'], name='direction')
    ds_probes.append(xr.concat([ds_ups, ds_downs], dim=direction))

probe = xr.DataArray(np.array(probe_designations), dims=['probe'], name='probe')
ds_full = xr.concat(ds_probes, dim=probe)

ds_full = ds_full.sel(probe = 'S')

manual_start = 0
manual_end = 750
if not magnum_access_fl:
    ds_full.max(dim='time').mean('direction')['current'].plot.line(x='sweep')
    ds_full.max(dim='time').mean('direction').isel(sweep=slice(manual_start,manual_end))['current'].plot.line(x='sweep')

    # Choose only the IVs in the static section
    ds_full = ds_full.isel(sweep=slice(manual_start,manual_end))

# Average across each sweep direction
sweep_avg_up = ds_full.sel(direction = 'up').mean('sweep')
sweep_avg_dn = ds_full.sel(direction = 'down').mean('sweep')

# Add in standard deviation of each bin as a new data variable 
sweep_avg_up = sweep_avg_up.assign({'d_current': ds_full.sel(direction = 'up').std('sweep')['current']})
sweep_avg_dn = sweep_avg_dn.assign({'d_current': ds_full.sel(direction = 'down').std('sweep')['current']})

print(ds_full)

999 999
999 999
<xarray.Dataset>
Dimensions:     (direction: 2, sweep: 750, time: 500)
Coordinates:
  * time        (time) float64 0.0 1e-05 2e-05 3e-05 ... 0.00497 0.00498 0.00499
  * direction   (direction) <U4 'up' 'down'
    probe       <U1 'S'
Dimensions without coordinates: sweep
Data variables:
    voltage     (direction, sweep, time) float64 -82.2 -83.35 ... -1.72 -1.757
    current     (direction, sweep, time) float64 -0.7797 -0.8003 ... 0.7366
    shot_time   (direction, sweep, time) float64 1.004 1.004 ... 8.499 8.499
    start_time  (direction, sweep) float64 1.004 1.014 1.024 ... 8.489 8.499


In [54]:
plt.figure()
plt.errorbar(sweep_avg_up['time'], sweep_avg_up['current'], yerr=sweep_avg_up['d_current'], 
             ecolor='silver')

plt.figure()
plt.errorbar(sweep_avg_dn['time'], sweep_avg_dn['current'], yerr=sweep_avg_dn['d_current'], 
             ecolor='silver')
plt.show()

In [55]:
sweep_avg_updn = ds_full.mean('direction').mean('sweep').assign({'d_current': ds_full.std('direction').std('sweep')['current']})
sweep_avg_updn = sweep_avg_updn.where(sweep_avg_updn.current <= 0, drop=True)
print(sweep_avg_updn)

sweep_avg_updn['current'].plot.line()

<xarray.Dataset>
Dimensions:     (time: 434)
Coordinates:
  * time        (time) float64 0.0 1e-05 2e-05 3e-05 ... 0.00431 0.00432 0.00433
    probe       <U1 'S'
Data variables:
    voltage     (time) float64 -82.12 -82.01 -81.79 ... -3.33 -3.315 -3.298
    current     (time) float64 -0.8166 -0.8153 -0.8149 ... -0.02187 -0.009908
    shot_time   (time) float64 4.753 4.753 4.753 4.753 ... 4.753 4.753 4.753
    start_time  (time) float64 4.751 4.751 4.751 4.751 ... 4.751 4.751 4.751
    d_current   (time) float64 0.02031 0.02521 0.02605 ... 0.01056 0.0106


In [43]:
sweep_avg_updn.to_netcdf('~/Data/highdens_test.nc')

In [2]:
sweep_avg_updn = xr.open_dataset('~/Data/highdens_test.nc')

In [9]:
plt.figure()
plt.plot(sweep_avg_up['voltage'], sweep_avg_up['current'], color='orange')
plt.plot(sweep_avg_dn['voltage'], sweep_avg_dn['current'], color='yellow')
plt.errorbar(sweep_avg_updn['voltage'], sweep_avg_updn['current'], yerr=sweep_avg_updn['d_current'], ecolor='silver')
plt.show()

# Multi-fitting

In [135]:
# concatenate the up and down sweeps together to cancel the (small) capacitance effect

# iv_data = ivd.IVData(np.concatenate([sweep_avg_up.where(sweep_avg_up.current < 0, drop=True)['voltage'].data, np.flip(sweep_avg_dn.where(sweep_avg_dn.current < 0)['voltage'].data)]), 
#                      -np.concatenate([sweep_avg_up.where(sweep_avg_up.current < 0)['current'].data, np.flip(sweep_avg_dn.where(sweep_avg_dn.current < 0)['current'].data)]),
#                      relative_t, 
#                      sigma=np.concatenate([sweep_avg_up.where(sweep_avg_up.current < 0)['d_current'].data, np.flip(sweep_avg_dn.where(sweep_avg_dn.current < 0)['d_current'].data)]), 
#                      estimate_error_fl=False)

iv_data = ivd.IVData(sweep_avg_updn['voltage'].data, 
                     -sweep_avg_updn['current'].data, 
                     sweep_avg_updn['time'].data, 
                     sigma=sweep_avg_updn['d_current'].data, estimate_error_fl=False)

# plt.figure()
# plt.plot(iv_data['V'], iv_data['I'])

starting_params = [0.69, 0.009, 1.12, +1]

full_iv_fitter = fts.FullIVFitter()
fit_data = full_iv_fitter.fit_iv_data(iv_data, initial_vals=starting_params)
fit_data.plot(show_fl=False)
# plt.errorbar(fit_data.raw_x, fit_data.raw_y, yerr=iv_data['sigma'], ecolor='silver')
# plt.plot(fit_data.raw_x, fit_data.fit_y, color='orange', label=r'')
# plt.plot(iv_data['V'], full_iv_fitter.fit_function(iv_data['V'], *starting_params), label='Start-param IV')
plt.legend()
plt.ylim([-.25, 1])

# fit_data = iv_data.multi_fit(plot_fl=True)
# fit_data_trim = iv_data_trim.multi_fit(plot_fl=True)

(-0.25, 1)

In [57]:
iv_data_corrected, fit_data_corrected = iv_data.gunn_fit(plot_fl=True)

In [29]:
# Try multifitting with a 3-parameter fit
simple_iv_fitter = fts.SimpleIVFitter()
iv_data.multi_fit(plot_fl=True, fitter=simple_iv_fitter)

RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

In [12]:
log_current = np.log(-iv_data['I'] + 0.8)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6))
ax1.errorbar(iv_data['V'], log_current, yerr=iv_data['sigma'], ecolor='silver', zorder=-2)
log_ssroi_1 = np.where((iv_data['V'] > -18) & (iv_data['V'] < -11))
log_ssroi_2 = np.where((iv_data['V'] > -40) & (iv_data['V'] < -22))
ax1.plot(iv_data['V'][log_ssroi_1], log_current[log_ssroi_1], color='orange')
ax1.plot(iv_data['V'][log_ssroi_2], log_current[log_ssroi_2], color='red')
ax11 = ax1.twinx()
ax11.plot(iv_data['V'], iv_data['I'], color='gray', zorder=-10)

sl_fitter = fts.StraightLineFitter()
fit_log_roi_1 = sl_fitter.fit(iv_data['V'][log_ssroi_1], log_current[log_ssroi_1])
fit_log_roi_2 = sl_fitter.fit(iv_data['V'][log_ssroi_2], log_current[log_ssroi_2])


ax2.plot(*fit_log_roi_1.get_raw_plottables(), color='silver')
ax2.plot(*fit_log_roi_1.get_fit_plottables(), label=r'T$_e$ = {:.2g} eV'.format(1/(fit_log_roi_1.get_param('m'))))
ax2.plot(*fit_log_roi_2.get_raw_plottables(), color='silver')
ax2.plot(*fit_log_roi_2.get_fit_plottables(), label=r'T$_e$ = {:.2g} eV'.format(1/(fit_log_roi_2.get_param('m'))))
ax2.legend()

plt.tight_layout()
plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


In [50]:
plt.figure()
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='r', zorder=10)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], ecolor='silver', label='Raw')
plt.legend()
plt.show()

In [49]:
#Try a single trim
iv_data_trim = ivd.IVData.non_contiguous_trim(iv_data, np.where(iv_data['I'] >= -0.4))
plt.figure()
plt.plot(iv_data_trim['V'], iv_data_trim['I'])
fit_data_trim = iv_data_trim.multi_fit()

plt.figure()
plt.plot(*fit_data.get_fit_plottables(), label=r'$T_e$ = {:.3g} eV'.format(fit_data.get_temp()), color='r', zorder=10)
plt.errorbar(iv_data['V'], iv_data['I'], yerr=iv_data[c.SIGMA], ecolor='silver', label='Raw')
plt.plot(iv_data['V'], fit_data_trim.fit_function(iv_data['V']), color='orange', label=r'$T_e$ = {:.3g} eV'.format(fit_data_trim.get_temp()), zorder=10)
plt.legend()
plt.ylim(-0.2, 1.3)
plt.show()

RuntimeError: Optimal parameters not found: The maximum number of function evaluations is exceeded.

# Averaged IV Figure

In [58]:
# Create new averaged iv figure
theta_perp = np.radians(9.95)

probe_selected = probe_L

A_coll_0 = probe_S.get_collection_area(theta_perp)
d_A_coll = np.abs(probe_S.get_collection_area(theta_perp + np.radians(0.8)) - A_coll_0)

v_f_fitted = fit_data.get_param('V_f')
# T_e_fitted = fit_data[c.ELEC_TEMP]
# a_fitted = fit_data[c.SHEATH_EXP]
# I_sat_fitted = fit_data[c.ION_SAT]

d_v_f_fitted = fit_data.get_param('V_f', errors_fl=True).error
# d_T_e_fitted = iv_data[c.ERROR_STRING.format(c.ELEC_TEMP)].values[0]
# d_a_fitted = iv_data[c.ERROR_STRING.format(c.SHEATH_EXP)].values[0]
# d_I_sat_fitted = iv_data[c.ERROR_STRING.format(c.ION_SAT)].values[0]

v_f_approx = - 3 * fit_data.get_temp()
d_v_f_approx = 0.05 * v_f_approx

v_f_approx_ts = - 3 * T_e_ts
d_v_f_approx_ts = 0.05 * v_f_approx_ts

c_s_fitted = lp.sound_speed(fit_data.get_temp(), gamma_i=1)
d_c_s_fitted = lp.d_sound_speed(c_s_fitted, fit_data.get_temp(), fit_data.get_temp(errors_fl=True).error)
n_e_fitted = lp.electron_density(fit_data.get_isat(), c_s_fitted, A_coll_0)
d_n_e_fitted = lp.d_electron_density(n_e_fitted, c_s_fitted, d_c_s_fitted, A_coll_0, d_A_coll, fit_data.get_isat(),
                                     fit_data.get_isat(errors_fl=True).error)

In [59]:
print(f"iv = averaged: \n"
      f"\t v_f = {v_f_fitted:.3g} +- {d_v_f_fitted:.1g} \n"
      f"\t T_e = {fit_data.get_temp():.3g} +- {fit_data.get_temp(errors_fl=True).error:.1g} \n"
      f"\t I_sat = {fit_data.get_isat():.3g} +- {fit_data.get_isat(errors_fl=True).error:.1g} \n"
      f"\t n_e = {n_e_fitted:.3g} +- {d_n_e_fitted:.1g} \n"
      f"\t a = {fit_data.get_sheath_exp():.3g} +- {fit_data.get_sheath_exp(errors_fl=True).error:.1g} \n"
      f"\t c_s = {c_s_fitted:.3g} +- {d_c_s_fitted:.1g} \n"
      f"\t A_coll = {A_coll_0:.3g} +- {d_A_coll:.1g} \n")

iv = averaged: 
	 v_f = -3.27 +- 0.001 
	 T_e = 1 +- 0.003 
	 I_sat = 0.676 +- 0.0003 
	 n_e = 1.11e+20 +- 7e+18 
	 a = 0.00763 +- 3e-05 
	 c_s = 1.39e+04 +- 2e+01 
	 A_coll = 2.75e-06 +- 2e-07 



In [61]:
I_f = probe_S.get_analytical_iv(fit_data.raw_x, v_f_fitted, theta_perp, fit_data.get_temp(), n_e_fitted,
                                print_fl=True)
I_ts = probe_S.get_analytical_iv(fit_data.raw_x, v_f_approx_ts, theta_perp, T_e_ts, n_e_ts,
                                 print_fl=True)

plt.figure()
plt.errorbar(fit_data.raw_x, fit_data.raw_y, yerr=fit_data.sigma,
             label='Raw IV', ecolor='silver', color='gray', zorder=-1)
# plt.plot(iv_data[c.RAW_X].tolist()[0], I_f, label='Analytical - measured', linestyle='dashed', linewidth=1, color='r')
plt.plot(fit_data.raw_x, fit_data.fit_y, color='blue', linewidth=1.2,
         label='Fit - ({:.2g}eV, {:.2g}m'.format(fit_data.get_temp(), n_e_fitted)+r'$^{-3}$)')
plt.plot(fit_data.raw_x, I_ts, linestyle='dashed', color='red',
         label='Analytical from TS - ({:.2g}eV, {:.2g}m'.format(T_e_ts, n_e_ts)+'$^{-3}$)')

plt.legend()
# plt.title('Comparison of analytical to measured IV curves for the small area probe')
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim([-0.01, 3.2])

a = 0.0014727166707731423, c_s = 13874.1470391269, lambda_d = 7.085019324735317e-07, I_0 = 0.675892287975617
a = 0.0007760949548473818, c_s = 14648.066898304953, lambda_d = 3.733676586981677e-07, I_0 = 2.864232327532151


(-0.01, 3.2)

# Playing with those pesky CSV files

The changes I made to the magopter prepare method seem to have brought the temperature and density down by a factor of 4, which is odd. I have an older csv full of fitted values which I will investigate in this segment.

In [105]:
fit_dfs = magopter.fit(load_fl=True)
print(fit_dfs[1].columns)
fit_dfs[1].index = fit_dfs[1].pop('Unnamed: 0')
fit_dfs[1].index.name = 'time'

fit_df_arrs = {}
for arr_column in ['raw_x', 'raw_y', 'fit_y', 'sigma']:
    fit_df_arrs[arr_column] = fit_dfs[1].pop(arr_column)
    
fit_xrds = fit_dfs[1].to_xarray()
print(fit_xrds)

Index(['Unnamed: 0', 'I_sat', 'T_e', 'V_f', 'a', 'chi2', 'd_I_sat', 'd_T_e',
       'd_V_f', 'd_a', 'fit_y', 'raw_x', 'raw_y', 'reduced_chi2', 'sigma'],
      dtype='object')
<xarray.Dataset>
Dimensions:       (time: 1016)
Coordinates:
  * time          (time) float64 1.008 1.038 1.043 1.048 ... 10.97 10.98 10.99
Data variables:
    I_sat         (time) float64 -1.129 -1.131 -1.132 ... 0.001015 0.00107
    T_e           (time) float64 9.113 9.258 9.361 ... 0.008513 0.07098 0.7068
    V_f           (time) float64 -4.629 -4.476 -4.392 ... -57.35 -62.21 -14.96
    a             (time) float64 2.416e-17 2.681e-17 ... 0.001731 3.881e-19
    chi2          (time) float64 7.649e+03 7.797e+03 7.244e+03 ... 342.2 847.3
    d_I_sat       (time) float64 0.009329 0.009579 ... 9.001e-05 6.458e-05
    d_T_e         (time) float64 0.1578 0.1601 0.1647 ... 1.585e+05 0.1216 1.102
    d_V_f         (time) float64 0.05592 0.05463 0.05558 ... 2.533 0.04892 1.451
    d_a           (time) float64 0.001922 0.

In [31]:
fit_df = fit_dfs[1]
print(fit_df.columns)

Index(['I_sat', 'T_e', 'V_f', 'a', 'chi2', 'd_I_sat', 'd_T_e', 'd_V_f', 'd_a',
       'reduced_chi2'],
      dtype='object')


In [112]:
fit_xrds = fit_xrds.isel(time=slice(0,650))

In [140]:
fig, (ax1, ax2) = plt.subplots(2)

ax1.plot(fit_df.index, fit_df['T_e'], label='Temperature')
ax1.plot(fit_df.index, -fit_df['I_sat'], label='Saturation Current')
ax1.plot(fit_xrds['time'], fit_xrds['T_e'], label='Temperature Trimmed')
ax1.errorbar(fit_xrds['time'], -fit_xrds['I_sat'], label='Saturation Current Trimmed')
ax1.set_ylim([0, 12])
ax1.legend()

# ax2.plot(fit_dfs[0]['raw_x'][12], fit_dfs[0]['raw_y'][12])
# ax2.plot(fit_dfs[0]['raw_x'][12], fit_dfs[0]['fit_y'][12])
# plt.show()

In [116]:
fit_ds_avg = fit_xrds.mean('time')
print(fit_ds_avg)
print(fit_ds_avg['V_f'].data)

<xarray.Dataset>
Dimensions:       ()
Data variables:
    I_sat         float64 -1.137
    T_e           float64 9.029
    V_f           float64 -4.195
    a             float64 3.843e-05
    chi2          float64 7.937e+03
    d_I_sat       float64 0.009095
    d_T_e         float64 0.1528
    d_V_f         float64 0.05436
    d_a           float64 0.001849
    reduced_chi2  float64 9.152
-4.194917512169531


In [117]:
A_coll_L = probe_L.get_collection_area(theta_perp)
d_A_coll_L = np.abs(probe_L.get_collection_area(theta_perp + np.radians(0.8)) - A_coll_L)


c_s_oldfitted = lp.sound_speed(fit_ds_avg['T_e'].data, gamma_i=1)
d_c_s_oldfitted = lp.d_sound_speed(c_s_oldfitted, fit_ds_avg['T_e'].data, fit_ds_avg['d_T_e'].data)
n_e_oldfitted = lp.electron_density(-fit_ds_avg['I_sat'].data, c_s_oldfitted, A_coll_L)
d_n_e_oldfitted = lp.d_electron_density(n_e_oldfitted, c_s_oldfitted, d_c_s_oldfitted, A_coll_L, d_A_coll_L, -fit_ds_avg['I_sat'].data,
                                        fit_ds_avg['d_I_sat'].data)
print(f"{n_e_oldfitted:.3g}")

print(f"iv = averaged: \n"
      f'\t v_f = {fit_ds_avg["V_f"].data:.3g} +- {fit_ds_avg["d_V_f"].data:.1g} \n'
      f"\t T_e = {(fit_ds_avg['T_e'].data):.3g} +- {fit_ds_avg['d_T_e'].data:.1g} \n"
      f"\t I_sat = {fit_ds_avg['I_sat'].data:.3g} +- {fit_ds_avg['d_I_sat'].data:.1g} \n"
      f"\t n_e = {n_e_oldfitted:.3g} +- {d_n_e_oldfitted:.1g} \n"
      f"\t a = {fit_ds_avg['a'].data:.3g} +- {fit_ds_avg['d_a'].data:.1g} \n"
      f"\t c_s = {c_s_oldfitted:.3g} +- {d_c_s_oldfitted:.1g} \n"
      f"\t A_coll = {A_coll_L:.3g} +- {d_A_coll_L:.1g} \n")

2.04e+19
iv = averaged: 
	 v_f = -4.19 +- 0.05 
	 T_e = 9.03 +- 0.2 
	 I_sat = -1.14 +- 0.009 
	 n_e = 2.04e+19 +- 1e+18 
	 a = 3.84e-05 +- 0.002 
	 c_s = 4.16e+04 +- 4e+02 
	 A_coll = 8.36e-06 +- 4e-07 



In [104]:
# I_f = probe_S.get_analytical_iv(fit_data.raw_x, v_f_fitted, theta_perp, fit_data.get_temp(), n_e_fitted,
#                                 print_fl=True)
I_oldf = probe_S.get_analytical_iv(fit_data.raw_x, fit_ds_avg['V_f'].data, theta_perp, fit_ds_avg['T_e'].data, n_e_oldfitted,
                                print_fl=True)
# I_ts = probe_S.get_analytical_iv(fit_data.raw_x, v_f_approx_ts, theta_perp, T_e_ts, n_e_ts,
#                                  print_fl=True)

plt.figure()
plt.errorbar(fit_data.raw_x, fit_data.raw_y, yerr=fit_data.sigma,
             label='Raw IV', ecolor='silver', color='gray', zorder=-1)
# plt.plot(iv_data[c.RAW_X].tolist()[0], I_f, label='Analytical - measured', linestyle='dashed', linewidth=1, color='r')
plt.plot(fit_data.raw_x, fit_data.fit_y, color='blue', linewidth=1.2,
         label='Fit - ({:.2g}eV, {:.2g}m'.format(fit_data.get_temp(), n_e_fitted)+r'$^{-3}$)')
plt.plot(fit_data.raw_x, I_ts, linestyle='dashed', color='red',
         label='Analytical from TS - ({:.2g}eV, {:.2g}m'.format(T_e_ts, n_e_ts)+'$^{-3}$)')
plt.plot(fit_data.raw_x, I_oldf, linestyle='dashed', color='orange',
         label='Old Fit - ({:.2g}eV, {:.2g}m'.format(fit_ds_avg['T_e'].data, n_e_oldfitted)+'$^{-3}$)')

plt.legend()
# plt.title('Comparison of analytical to measured IV curves for the small area probe')
plt.xlabel(r'$V_p$ / V')
plt.ylabel(r'$I$ / A')
plt.ylim([-0.01, 3.2])

a = 0.005946861913554874, c_s = 35491.37632371556, lambda_d = 2.8609461965925557e-06, I_0 = 0.6938887295173395


(-0.01, 3.2)

In [126]:
fit_df_arrs_concatted = pd.concat(list(fit_df_arrs.values()), axis=1)

In [145]:
np.array(fit_df_arrs_concatted.iloc[350]['raw_x'])[0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-145-ce1ffd612af3>", line 1, in <module>
    os.chdir('~')
FileNotFoundError: [Errno 2] No such file or directory: '~'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerfra

FileNotFoundError: [Errno 2] No such file or directory: '~'